In [1]:
import sys
sys.path.insert(0, "../..")

In [21]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD, SeparateAcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch():
    model.train()
    param_hist = []
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        optimizer.step()
    return torch.tensor(np.mean(param_hist, 0))

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Conv2d(1, 32, 3),
    nn.ReLU(),
    nn.Conv2d(32, 32, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3),
    nn.ReLU(),
    nn.Conv2d(64, 64, 3),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(4*4*64, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (6): ReLU()
  (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (8): ReLU()
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=1024, out_features=128, bias=True)
  (12): ReLU()
  (13): Linear(in_features=128, out_features=10, bias=True)
  (14): LogSoftmax(dim=-1)
)

In [7]:
initial_state = deepcopy(model.state_dict())

In [8]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch")
loss_fn = nn.NLLLoss()

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1004, validation loss: 2.3019
Epoch 2



Validation accuracy: 0.1004, validation loss: 2.3014
Epoch 3



Validation accuracy: 0.1019, validation loss: 2.3010
Epoch 4


...


Validation accuracy: 0.9352, validation loss: 0.2075
Epoch 27



Validation accuracy: 0.9437, validation loss: 0.1852
Epoch 28



Validation accuracy: 0.9462, validation loss: 0.1747
Epoch 29



Validation accuracy: 0.9492, validation loss: 0.1677
Epoch 30



Validation accuracy: 0.9523, validation loss: 0.1573


In [10]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9558, device='cuda:0'), tensor(0.1456, device='cuda:0'))



Valid: (tensor(0.9523, device='cuda:0'), tensor(0.1573, device='cuda:0'))


In [11]:
optimizer.accelerate()

In [12]:
optimizer.store_parameters()

In [13]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9580, device='cuda:0'), tensor(0.1371, device='cuda:0'))



Valid: (tensor(0.9563, device='cuda:0'), tensor(0.1487, device='cuda:0'))


## Epoch average

In [14]:
model.load_state_dict(initial_state)
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch_avg")

In [15]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1004, validation loss: 2.3019
Epoch 2



Validation accuracy: 0.1004, validation loss: 2.3014
Epoch 3



Validation accuracy: 0.1016, validation loss: 2.3010
Epoch 4



Validation accuracy: 0.1971, validation loss: 2.3005
Epoch 5


...


Validation accuracy: 0.9375, validation loss: 0.1999
Epoch 27



Validation accuracy: 0.9426, validation loss: 0.1866
Epoch 28



Validation accuracy: 0.9470, validation loss: 0.1742
Epoch 29



Validation accuracy: 0.9481, validation loss: 0.1683
Epoch 30



Validation accuracy: 0.9533, validation loss: 0.1562


In [16]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9565, device='cuda:0'), tensor(0.1444, device='cuda:0'))



Valid: (tensor(0.9533, device='cuda:0'), tensor(0.1562, device='cuda:0'))


In [17]:
optimizer.accelerate()

In [18]:
optimizer.store_parameters()

In [19]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9628, device='cuda:0'), tensor(0.1218, device='cuda:0'))



Valid: (tensor(0.9587, device='cuda:0'), tensor(0.1338, device='cuda:0'))


## Split + epoch

In [22]:
model.load_state_dict(initial_state)
optimizer = SeparateAcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch")

In [23]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1


/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/torch/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



Validation accuracy: 0.1004, validation loss: 2.3019
Epoch 2



Validation accuracy: 0.1004, validation loss: 2.3014
Epoch 3



Validation accuracy: 0.1018, validation loss: 2.3010
Epoch 4


...


Validation accuracy: 0.9362, validation loss: 0.2044
Epoch 27



Validation accuracy: 0.9434, validation loss: 0.1852
Epoch 28



Validation accuracy: 0.9446, validation loss: 0.1802
Epoch 29



Validation accuracy: 0.9465, validation loss: 0.1739
Epoch 30



Validation accuracy: 0.9512, validation loss: 0.1635


In [24]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9547, device='cuda:0'), tensor(0.1493, device='cuda:0'))



Valid: (tensor(0.9512, device='cuda:0'), tensor(0.1635, device='cuda:0'))


In [25]:
optimizer.accelerate()

In [26]:
optimizer.store_parameters()

In [27]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9479, device='cuda:0'), tensor(0.1777, device='cuda:0'))



Valid: (tensor(0.9449, device='cuda:0'), tensor(0.1967, device='cuda:0'))


## Split + epoch average

In [28]:
model.load_state_dict(initial_state)
optimizer = SeparateAcceleratedSGD(model.parameters(), 1e-3, k=10, mode="epoch_avg")

In [29]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    train_epoch()
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")

Epoch 1



Validation accuracy: 0.1004, validation loss: 2.3019
Epoch 2



Validation accuracy: 0.1004, validation loss: 2.3014
Epoch 3



Validation accuracy: 0.1020, validation loss: 2.3010
Epoch 4



Validation accuracy: 0.1983, validation loss: 2.3005
Epoch 5


...


Validation accuracy: 0.9364, validation loss: 0.2041
Epoch 27



Validation accuracy: 0.9395, validation loss: 0.1966
Epoch 28



Validation accuracy: 0.9389, validation loss: 0.1960
Epoch 29



Validation accuracy: 0.9507, validation loss: 0.1623
Epoch 30



Validation accuracy: 0.9505, validation loss: 0.1581


In [30]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9547, device='cuda:0'), tensor(0.1490, device='cuda:0'))



Valid: (tensor(0.9505, device='cuda:0'), tensor(0.1581, device='cuda:0'))


In [31]:
optimizer.accelerate()

In [32]:
optimizer.store_parameters()

In [33]:
print("Train:", validation(model, train_loader))
print("Valid:", validation(model, valid_loader))


Train: (tensor(0.9569, device='cuda:0'), tensor(0.1442, device='cuda:0'))



Valid: (tensor(0.9518, device='cuda:0'), tensor(0.1597, device='cuda:0'))
